Load Dataset

In [232]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from collections import Counter

In [233]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)

Model

In [234]:
def entropy(y):
    ps = np.bincount(y) / len(y)
    return -np.sum( [ p * np.log(p) for p in ps if p> 0])

In [235]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None
    
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_feats = n_feats
        self.root = None
        
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
        
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        if ((depth >= self.max_depth) or (n_samples < self.min_samples_split) or (n_labels == 1)):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:,best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)
        
        
    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        
        for feat_idx in feat_idxs:
            X_column = X[:,feat_idx]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
                    
        return split_idx, split_thresh
    
    def _information_gain(self, y, current_feat, threshold):
        # parent
        parent_entropy = entropy(y)
        
        # split
        left_idxs, right_idxs = self._split(current_feat, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # weighted avg child
        left_weight = len(left_idxs) / len(y)
        right_weight = len(right_idxs) / len(y)
        child_entropy = left_weight * entropy(y[left_idxs]) + right_weight * entropy(y[right_idxs])
        
        # ig
        ig = parent_entropy - child_entropy
        return ig
        
    def _split(self, current_feat, threshold):
        left_idxs = np.argwhere(current_feat <= threshold).flatten()
        right_idxs = np.argwhere(current_feat > threshold).flatten()
        return left_idxs, right_idxs
    
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

Testing

In [236]:
def accuracy(y_true, y_pred):
    acc = np.sum(y_true == y_pred) / len(y_pred)
    return acc

In [237]:
DC = DecisionTree()
DC.fit(X_train, y_train)
predictions = DC.predict(X_test)
accuracy(y_test, predictions)

0.9649122807017544